In [ ]:
import time
from datetime import datetime
from thredds_crawler.crawl import Crawl
from dask.diagnostics import ProgressBar
import xarray as xr

In [ ]:
def get_datasource_ref(
    url="https://thredds.met.no/thredds/catalog/metusers/heikok/ash/krisuvik/catalog.html",
    regex=".*eemep_hourInst*",
):
    results = {}
    c = Crawl(url, select=[regex])
    for i in c.datasets:
        dates, times = i.name.split("_")[-1].replace(".nc", "").split("T")
        datetime_object = datetime.strptime(f"{dates} {times}", "%Y%m%d %H%M%S")
        results[datetime_object] = {}
        for j in i.services:
            if j["service"] == "OPENDAP":
                results[datetime_object]["url"] = j["url"]
    return results

In [ ]:
catalog_datasources = get_datasource_ref()
catalog_datasources

In [ ]:
latest_cached_index = max(list(catalog_datasources.keys()))
nc_url = catalog_datasources[latest_cached_index]["url"]
latest_cached_index, nc_url

In [ ]:
eemep = xr.open_dataset(nc_url, chunks={"time": 48})

In [ ]:
mask = eemep["COLUMN_ASH_kmax"][:].where(eemep.COLUMN_ASH_kmax >= 200000)

In [ ]:
with ProgressBar():
    mask = mask.compute()

In [ ]:
# coarsen the dataset for quick testing of plot options
#mask = mask.coarsen(lon=3, boundary='pad').mean().coarsen(lat=3, boundary="trim").mean()

In [ ]:
mask

In [ ]:
import xarray as xr
import sys
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import warnings 

warnings.filterwarnings('ignore', '.*tight_layout*',)
warnings.filterwarnings('ignore', '.*GEBCO_LATEST*',)

#data = data_array["COLUMN_ASH_kmax"][:].where(data_array.COLUMN_ASH_kmax >= 200000)

""" Plotting routines
        the following routines require a rewriting, 
        TODO: 
            add logging
            expose more parameters as options
            building a proper command line argument parsing        
"""


def simpler_plot(val, bbox, output):
    # create figure
    fig = plt.figure(figsize=(8, 6), dpi=100)
    ax = plt.axes(projection=crs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.stock_img()
    ax.imshow(
        val,
        extent=(bbox),
    )
    plt.savefig(output)
    plt.close(fig)
    plt.close('all')


def get_time_image(data, output=None, dpi=60):
    fig = plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=crs.PlateCarree())
    ax.stock_img()
    # ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_wms(
        wms='https://www.gebco.net/data_and_products/gebco_web_services/web_map_service/mapserv?request=getcapabilities&service=wms&version=1.3.0',
        layers=['GEBCO_LATEST_2'])
    gl = ax.gridlines(crs=crs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.3, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.left_labels = True
    gl.bottom_labels = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.ylabel_style = {'size': 11, 'color': 'black'}
    gl.xlabel_style = {'size': 11, 'color': 'black'}
    ax.set_extent([-26, 32, 40, 80], crs=crs.PlateCarree())
    # ax.set_extent([3, 32, 54, 80], crs=crs.PlateCarree())
    # ax.set_aspect('equal', 'box')
    # data.plot(cmap=False) # (cmap=plt.cm.RdBu_r) #dir(plt.cm)
    im = data.plot.pcolormesh(add_colorbar=False, robust=True)
    # plt.pcolor(X, Y, f(data), cmap=cm, vmin=-4, vmax=4)
    cb = plt.colorbar(im, orientation="vertical", pad=0.05)
    cb.set_label(label=f"{data.attrs['long_name'].replace('_', ' ')} [$\mu g/m^{2}$]", size='large', weight='bold')
    # cb.set_label(label=f"{data.attrs['long_name'].replace('_', ' ')} [ug/m2]", size='large', weight='bold')
    cb.ax.tick_params(labelsize='large')
    plt.xlabel("Longitude")
    plt.xlabel("Latitude")
    if output:
        plt.savefig(output, dpi=dpi)
    else:
        plt.show()
    plt.close(fig)
    plt.close('all')


def get_time_sequence_image(data, output=None, dpi=60):
    map_proj = crs.PlateCarree()
    fig = plt.figure()
    p = data.plot(transform=crs.PlateCarree(),
                  robust=True,
                  col='time', col_wrap=3,  # multiplot settings
                  aspect=data.lon.shape[0] / data.lat.shape[0],  # for a sensible figsize
                  subplot_kws={'projection': map_proj})  # the plot's projection

    # bottom_idx is based on the subplot arrangements and the number of time-slice
    bottom_idx = [9, 10, 11]

    # We have to set the map's options on all four axes
    for i, ax in enumerate(p.axes.flat):
        ax.stock_img()
        ax.coastlines()
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        gl = ax.gridlines(crs=crs.PlateCarree(), draw_labels=True,
                          linewidth=1, color='gray', alpha=0.3, linestyle='--')
        # the following are deprecated in newer cartopy
        # gl.xlabels_top = False
        # gl.ylabels_right = False
        # gl.ylabels_left = True
        # if i not in bottom_idx:
        #     gl.xlabels_bottom = False
        # else:
        #     gl.xlabels_bottom = True
        #
        gl.top_labels = False
        gl.right_labels = False
        gl.left_labels = True
        if i not in bottom_idx:
            gl.bottom_labels = False
        else:
            gl.bottom_labels = True
        #
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.ylabel_style = {'size': 11, 'color': 'black'}
        gl.xlabel_style = {'size': 11, 'color': 'black'}
        ax.set_extent([-26, 32, 40, 80])
        ax.set_aspect('equal', 'box')

    if output:
        plt.savefig(output, dpi=dpi)
    else:
        plt.show()
    plt.close(fig)
    plt.close('all')

In [ ]:
get_time_image(mask[1])

In [ ]:
image_pages = [[0, 12], [12, 24], [24, 36], [36, 48]]
for slice in image_pages:
    get_time_sequence_image(mask[slice[0]:slice[1]])